In [2]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')
import pystan
import scipy.stats as stats
pd.set_option('max_rows', 1000)
pd.set_option('max_columns', 50)
from sqlalchemy import create_engine
import psycopg2 as pq
golf_db = 'postgresql://localhost:5432/golf'

In [1]:
def exec_sql(sqlTxt, result=False):  
    with pq.connect(golf_db) as conn:
        with conn.cursor() as cursor:
            cursor.execute(sqlTxt) 
            if result:
                return [row for row in cursor] 
        
def pd_from_sql(sqlTxt):
    with pq.connect(golf_db) as conn:
        return pd.read_sql_query(sqlTxt, conn)
    
def np_to_sql(arr, table):
    df = pd.DataFrame(arr)
    with pq.connect(golf_db) as conn:
        df.to_sql(table, conn, if_exists='append')
        
def np_from_sql(sqlTxt):
    return pd_from_sql(sqlTxt).as_matrix()

In [17]:
sqlTxt = '''
select player_id, permanent_tournament_id, round, score from scores_view where player_id in (
    select player_id from stats_view where year = 2014 limit 100
)
'''
scores = pd_from_sql(sqlTxt)

In [31]:
player_ids = scores['player_id'].unique()
num_players = len (player_ids)
player_map = dict(zip(player_ids, range(1,num_players + 1)))
tournament_ids = scores['permanent_tournament_id']
num_tournaments = len (tournament_ids)
tournament_map = dict(zip(tournament_ids, range(1, num_tournaments + 1)))

In [32]:
datum = sorted([[player_map[i[0]], tournament_map[i[1]], i[3]] for i in scores], key = lambda (el): el[0])
p, t, y = zip(*datum)
rounds = np.zeros([100])
for s in p:
    scores[s-1] += 1
scores = scores.tolist()

KeyError: 'p'